In [1]:
import warnings
warnings.filterwarnings('ignore')
from pymongo import MongoClient
import pandas as pd
import numpy as np
# Requests sends and recieves HTTP requests.
import requests
# Beautiful Soup parses HTML documents in python.
from bs4 import BeautifulSoup

In [2]:
client = MongoClient('localhost', 27017)
db = client.tripadvisor_hon_eats_reviews
pages = db.pages

In [3]:
# pages.count_documents({})
# ->15,903 documents

In [78]:
soup = BeautifulSoup(pages.find_one()['html'], features="html.parser")

In [5]:
print(soup)

<!DOCTYPE html>
<html lang="en" xmlns:og="http://opengraphprotocol.org/schema/"><head><meta content="text/html; charset=utf-8" http-equiv="content-type"/><link href="https://static.tacdn.com/favicon.ico?v2" id="favicon" rel="icon" type="image/x-icon"/><link color="#000000" href="https://static.tacdn.com/img2/brand_refresh/application_icons/mask-icon.svg" rel="mask-icon" sizes="any"/><meta content="#34e0a1" name="theme-color"/><meta content="telephone=no" name="format-detection"/><script type="text/javascript">window.taRollupsAreAsync = true;</script><link crossorigin="" href="https://static.tacdn.com/css2/webfonts/TripSans/TripSans.css?v1.002" rel="stylesheet"/><title>HANKS CAFE HONOLULU - Downtown Honolulu - Restaurant Reviews &amp; Phone Number - Tripadvisor</title><meta content="TripAdvisor" property="al:ios:app_name"/><meta content="284876795" property="al:ios:app_store_id"/><meta content="284876795" name="twitter:app:id:ipad" property="twitter:app:id:ipad"/><meta content="28487679

In [6]:
def explode(df, lst_cols, fill_value=''):
    # make sure `lst_cols` is a list
    if lst_cols and not isinstance(lst_cols, list):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)

    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()

    if (lens > 0).all():
        # ALL lists in cells aren't empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, df[lst_cols[0]].str.len())
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .loc[:, df.columns]
    else:
        # at least one list in cells is empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, df[lst_cols[0]].str.len())
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .append(df.loc[lens==0, idx_cols]).fillna(fill_value) \
          .loc[:, df.columns]

In [7]:
listing_details = soup.find("div", {"id": "taplc_detail_overview_cards_0"})
listing_details

<div class="ppr_rup ppr_priv_detail_overview_cards" data-placement-name="detail_overview_cards" id="taplc_detail_overview_cards_0"><div class="react-container" data-component="@ta/restaurants.detail-overview-cards" data-component-props="page-manifest" id="component_36"><div class="restaurants-detail-overview-cards-DetailOverviewCards__cardBackground--34yTY" data-tab="TABS_OVERVIEW"><div class="ui_columns"><div class="restaurants-detail-overview-cards-DetailOverviewCards__cardColumn--1BmXT ui_column is-12-mobile is-4-desktop"><div class="restaurants-detail-overview-cards-DetailOverviewCards__wrapperDiv--1Dfhf"><div><h2>Ratings and reviews</h2><div class="restaurants-detail-overview-cards-RatingsOverviewCard__primaryRatingRow--VhEsu restaurants-detail-overview-cards-RatingsOverviewCard__cx_brand_refresh_phase2--1d7i-"><span class="restaurants-detail-overview-cards-RatingsOverviewCard__overallRating--nohTl">5.0<!-- --> </span><span class="restaurants-detail-overview-cards-RatingsOverviewC

In [34]:
# url
url = soup.find("link", {"rel": "alternate", "hreflang": "en"})['href']
print(url)

https://www.tripadvisor.com/Restaurant_Review-g60982-d12153726-Reviews-or60-Maui_Brewing_Company-Honolulu_Oahu_Hawaii.html


In [62]:
# top_details
top_details = soup.find("div", {"id": "taplc_top_info_0"})
top_details = top_details.findAll("a", {"class": "restaurants-detail-top-info-TopInfo__tagLink--2LkIo"})
top_details =[top_detail.getText() for top_detail in top_details]
top_details = ' | '.join(top_details)

# main_details
main_details = soup.find("div", {"id": "taplc_details_card_0"})

# about
about = main_details.find("div", {"class": "restaurants-details-card-DesktopView__desktopAboutText--1VvQH"}).getText()

# tags
tags = main_details.findAll("div", {"class": "restaurants-details-card-TagCategories__tagText--Yt3iG"})
tags = [tag.getText() for tag in tags]
tags = ' | '.join(tags)

'$$ - $$$ | American | Bar | Pub'

In [86]:
# more_details
more_details = soup.find("div", {"id": "taplc_detail_overview_cards_0"})
more_details = more_details.findAll("div", {"class": "restaurants-detail-overview-cards-DetailsSectionOverviewCard__tagText--1OH6h"})
more_details = [more_detail.getText() for more_detail in more_details]
more_details = ' | '.join(more_details)
more_details

'Breakfast, Lunch, Dinner'

In [93]:
def get_curr_page_info(soup):
    # restaurant_name
    restaurant_name = soup.find("h1", {"class": "header heading masthead masthead_h1"}).getText()
#     print(restaurant_name) #<-debug

    # description
    description = soup.find("meta", {"name": "description"})['content']
    
    # url
    url = soup.find("link", {"rel": "alternate", "hreflang": "en"})['href']
    
    # overall listing info
    listing_details = soup.find("div", {"id": "taplc_detail_overview_cards_0"})
    
    # top_details
    top_details = soup.find("div", {"id": "taplc_top_info_0"})
    top_details = top_details.findAll("a", {"class": "restaurants-detail-top-info-TopInfo__tagLink--2LkIo"})
    top_details =[top_detail.getText() for top_detail in top_details]
    top_details = ' | '.join(top_details)

    # main_details
    main_details = soup.find("div", {"id": "taplc_details_card_0"})

    # about
    try:
        about = main_details.find("div", {"class": "restaurants-details-card-DesktopView__desktopAboutText--1VvQH"}).getText()
    except AttributeError:
        about = np.nan
    
    # tags
    tags = main_details.findAll("div", {"class": "restaurants-details-card-TagCategories__tagText--Yt3iG"})
    tags = [tag.getText() for tag in tags]
    tags = ' | '.join(tags)
    
    # more_details
    more_details = soup.find("div", {"id": "taplc_detail_overview_cards_0"})
    more_details = more_details.findAll("div", {"class": "restaurants-detail-overview-cards-DetailsSectionOverviewCard__tagText--1OH6h"})
    more_details = [more_detail.getText() for more_detail in more_details]
    more_details = ' | '.join(more_details)

    # overall_rating
    try: 
        overall_rating = listing_details.find("span", {"class": "restaurants-detail-overview-cards-RatingsOverviewCard__overallRating--nohTl"})
        overall_rating = float(overall_rating.getText()[:3])
    except AttributeError:
        overall_rating = np.nan
        
    # more overall ratings (food, service, value, atmosphere)
    more_overall_rating_types = listing_details.findAll("span", {"class": "restaurants-detail-overview-cards-RatingsOverviewCard__ratingText--1P1Lq"})
    more_overall_rating_types = [rating.getText() for rating in more_overall_rating_types]

    more_overall_ratings_raw = listing_details.findAll("span", {"class": "ui_bubble_rating"})[1:]
    more_overall_ratings = [int(str(rating)[37:39])/10 for rating in more_overall_ratings_raw]

    more_overall_ratings_dict = {}
    for i in range(len(more_overall_rating_types)):
        more_overall_ratings_dict[more_overall_rating_types[i]] = more_overall_ratings[i]

    try: 
        food_rating = more_overall_ratings_dict['Food']
    except KeyError:
        food_rating = np.nan
    try: 
        service_rating = more_overall_ratings_dict['Service']
    except KeyError:
        service_rating = np.nan
    try: 
        value_rating = more_overall_ratings_dict['Value']
    except KeyError:
        value_rating = np.nan
    try: 
        atmosphere_rating = more_overall_ratings_dict['Atmosphere']
    except KeyError:
        atmosphere_rating = np.nan
        
    # num_reviews
    try: 
        num_reviews = listing_details.find("a", {"class": "restaurants-detail-overview-cards-RatingsOverviewCard__ratingCount--DFxkG"})
        num_reviews = int(num_reviews.getText().replace(',', '').split(' ')[0])
    except AttributeError:
        num_reviews = np.nan
    
    # ranking
    try:
        ranking = listing_details.findAll("div", {"class": "restaurants-detail-overview-cards-RatingsOverviewCard__ranking--17CmN"})
        ranking = ranking[-1].getText()
    except IndexError:
        ranking = np.nan
        
    # location info
    location_info = listing_details.findAll("span", {"class": "restaurants-detail-overview-cards-LocationOverviewCard__detailLinkText--co3ei"})

    # address
    try:
        address = location_info[0].getText()
    except IndexError:
        address = np.nan

    # location
    try: 
        location = location_info[1]
        location = location.findAll("div")[-1].getText()
    except IndexError:
        location = np.nan
        
    # image_url
    try: 
        image_urls = soup.find("div", {"class": "mosaic_photos"})
        image_url = image_urls.find("img", {"class": "basicImg"})['data-lazyurl']
    except TypeError:
        image_url = ''

    # user_names
    user_names = []
    user_names_raw = soup.findAll("div", {"class": "info_text pointer_cursor"})
    for name in user_names_raw:
        user_names.append(name.getText())

    # bubble_ratings
    bubble_ratings = []
    reviews = soup.find("div", {"class": "listContainer"})
    bubble_ratings_raw = reviews.findAll("span", {"class": "ui_bubble_rating"})
    for rating in bubble_ratings_raw:
        bubble_ratings.append(int(str(rating)[37:39])/10)

    # review_contents
    review_contents = []
    review_contents_raw = reviews.findAll("p", {"class": "partial_entry"})
    for review in review_contents_raw:
        review_contents.append(review.getText().replace('...More', ''))

    return [restaurant_name, description, url, top_details, about, tags, more_details, 
            overall_rating, food_rating, service_rating, value_rating, atmosphere_rating, 
            num_reviews, ranking, address, location, image_url, user_names, bubble_ratings, review_contents]

In [94]:
get_curr_page_info(soup = BeautifulSoup(pages.find_one()['html'], features="html.parser"))

['Hanks Cafe Honolulu',
 'Hanks Cafe Honolulu, Honolulu: See 5 unbiased reviews of Hanks Cafe Honolulu, rated 5 of 5 on Tripadvisor and ranked #1,270 of 2,222 restaurants in Honolulu.',
 'https://www.tripadvisor.com/Restaurant_Review-g60982-d4685440-Reviews-Hanks_Cafe_Honolulu-Honolulu_Oahu_Hawaii.html',
 '',
 nan,
 '',
 'Breakfast, Lunch, Dinner',
 5.0,
 nan,
 nan,
 nan,
 nan,
 5,
 '#943 of 1,581 Restaurants in Honolulu',
 '1038 Nuuanu Ave, Honolulu, Oahu, HI 96817-5117',
 '0.1 miles from Downtown Honolulu',
 '',
 ['waynen95', 'Maria H', 'RowerChicago', 'Kim J', 'cg148777'],
 [4.0, 5.0, 5.0, 5.0, 5.0],
 ['a small pub in Chinatown with great service and great live music upstairs. Service was friendly and efficient',
  'Friendly and a good selection of beers and. Liquors. Nice crowd.crowdSay sports bar.Say mahalo to Dave at the bar.',
  'Very casual fun spot for all kinds of hot dogs and burgers. Order at counter, short wait as food prepared to order. Great outside seating in up and cha

In [95]:
info = []
for page in pages.find({}).limit(50):
    soup = BeautifulSoup(page['html'], features="html.parser")
    curr_page_info = get_curr_page_info(soup)
    info.append(curr_page_info)
info_df = pd.DataFrame(np.array(info),
                   columns=['restaurant_name', 'description', 'url', 'top_details', 'about', 'tags', 
                            'more_details', 'overall_rating', 'food_rating', 'service_rating', 
                            'value_rating', 'atmosphere_rating', 'num_reviews', 'ranking', 
                            'address', 'location', 'image_url', 'user_names', 
                            'bubble_ratings', 'review_contents'])

In [96]:
info_df.head(50)

,restaurant_name,description,url,top_details,about,tags,more_details,overall_rating,food_rating,service_rating,value_rating,atmosphere_rating,num_reviews,ranking,address,location,image_url,user_names,bubble_ratings,review_contents
0,Hanks Cafe Honolulu,"Hanks Cafe Honolulu, Honolulu: See 5 unbiased ...",https://www.tripadvisor.com/Restaurant_Review-...,,NaN,,"Breakfast, Lunch, Dinner",5,NaN,NaN,NaN,NaN,5,"#943 of 1,581 Restaurants in Honolulu","1038 Nuuanu Ave, Honolulu, Oahu, HI 96817-5117",0.1 miles from Downtown Honolulu,,"[waynen95, Maria H, RowerChicago, Kim J, cg148...","[4.0, 5.0, 5.0, 5.0, 5.0]",[a small pub in Chinatown with great service a...
1,"Bethel Union, Honolulu","Bethel Union, Honolulu: See 9 unbiased reviews...",https://www.tripadvisor.com/Restaurant_Review-...,$$ - $$$,NaN,,Reservations,4,NaN,NaN,NaN,NaN,9,"#832 of 1,581 Restaurants in Honolulu","1115 Bethel St, Honolulu, Oahu, HI 96813-2202",0.1 miles from Downtown Honolulu,https://media-cdn.tripadvisor.com/media/photo-...,"[28franniel, LW811055, cindyq2, grantkidani, v...","[4.0, 4.0, 5.0, 4.0, 2.0, 5.0, 5.0]",[Our son took us to this new and hip spot to t...
2,"Cafe Anasia, Honolulu","Cafe Anasia, Honolulu: See 5 unbiased reviews ...",https://www.tripadvisor.com/Restaurant_Review-...,$$ - $$$ | Vietnamese,NaN,,"Vietnamese | Lunch, Dinner",4.5,NaN,NaN,NaN,NaN,5,"#903 of 1,581 Restaurants in Honolulu","2227 S Beretania St, Honolulu, Oahu, HI 96826-...",NaN,https://media-cdn.tripadvisor.com/media/photo-...,"[Amy M, EmmJoans, Craig O, Tim K]","[5.0, 5.0, 5.0, 4.0]",[This is an unexpected find. Great bar atmosph...
3,"Starbucks, Honolulu","Starbucks, Honolulu: See 8 unbiased reviews of...",https://www.tripadvisor.com/Restaurant_Review-...,$$ - $$$,NaN,,"Breakfast, Lunch, Dinner",4.5,NaN,NaN,NaN,NaN,8,"#749 of 1,581 Restaurants in Honolulu","949 Auahi St, Honolulu, Oahu, HI 96814",1.1 miles from Downtown Honolulu,https://media-cdn.tripadvisor.com/media/photo-...,"[vrite, jujuju123, tryblueocean, bombguy64, ga...","[4.0, 5.0, 5.0, 5.0, 4.0]",[Super busy Starbucks location but efficient a...
4,"La Vela Wine & Spa, Honolulu","La Vela Wine & Spa, Honolulu: See 2 unbiased r...",https://www.tripadvisor.com/Restaurant_Review-...,,NaN,,,5,NaN,NaN,NaN,NaN,2,"#1,040 of 1,581 Restaurants in Honolulu","2375 Ala Wai Blvd Waikiki Sand Villa Hotel, Ho...",Waikiki,https://media-cdn.tripadvisor.com/media/photo-...,"[bobika, Dianne W]","[5.0, 5.0]","[Great ambiance, wine and delicious salmon on ..."
5,"Golden City Restaurant, Honolulu","Golden City Restaurant, Honolulu: See 4 unbias...",https://www.tripadvisor.com/Restaurant_Review-...,$,NaN,,"Lunch, Dinner",4,NaN,NaN,NaN,NaN,4,"#1,238 of 1,581 Restaurants in Honolulu","1418 N School St, Honolulu, Oahu, HI 96817-1914",1.5 miles from Downtown Honolulu,https://media-cdn.tripadvisor.com/media/photo-...,"[116kristenf, Derek P, bf96819, travelmeisteren]","[3.0, 5.0, 4.0, 3.0]",[Went to the gas station next door and thought...
6,"Chinatown Kitchen, Honolulu","Chinatown Kitchen, Honolulu: See 3 unbiased re...",https://www.tripadvisor.com/Restaurant_Review-...,,NaN,,"Breakfast, Lunch, Dinner",4,NaN,NaN,NaN,NaN,3,"#1,183 of 1,581 Restaurants in Honolulu","119 N Hotel St Chinatown, Honolulu, Oahu, HI 9...",0.2 miles from Downtown Honolulu,https://media-cdn.tripadvisor.com/media/photo-...,"[ToroFarm, 221simont]","[5.0, 4.0]",[We ate at this mom and pop restaurant in Chin...
7,"Vima Cafe, Honolulu","Vima Cafe, Honolulu: See unbiased reviews of V...",https://www.tripadvisor.com/Restaurant_Review-...,,NaN,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1114 Fort Street Mall, Honolulu, Oahu, HI 9681...",0.1 miles from Downtown Honolulu,,[],[],[]
8,"Hunan Cuisine, Honolulu","Hunan Cuisine, Honolulu: See 14 unbiased revie...",https://www.tripadvisor.com/Restaurant_Review-...,$$ - $$$ | Chinese | Asian | Szechuan,NaN,,"Chinese, Asian | Lunch, Dinner | Seating, Tabl...",4,4,4,4,NaN,14,"#977 of 1,581 Restaurants in Honolulu","53 N Beretania St, Honolulu, Oahu, HI 9

In [92]:
info_df['top_details'][13]

'$$ - $$$ | American | Cafe | Vegetarian Friendly'

In [29]:
info_df.set_index('url')\
              .apply(lambda x: x.apply(pd.Series).stack())\
              .reset_index(level=1, drop=True).reset_index()

,url,restaurant_name,description,overall_rating,food_rating,service_rating,value_rating,atmosphere_rating,num_reviews,ranking,address,location,image_url,user_names,bubble_ratings,review_contents
0,https://www.tripadvisor.com/Restaurant_Review-...,"The Cream Pot, Honolulu","The Cream Pot, Honolulu: See 468 unbiased revi...",4.0,4.0,3.5,3.0,4.0,468.0,"#188 of 1,581 Restaurants in Honolulu","444 Niu St, Honolulu, Oahu, HI 96815-1830",Waikiki,https://media-cdn.tripadvisor.com/media/photo-...,y0gapam,5.0,"The most amazing soufflé pancakes & omelettes,..."
1,https://www.tripadvisor.com/Restaurant_Review-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vansancopp,4.0,For all the diners who dream of starring in a ...
2,https://www.tripadvisor.com/Restaurant_Review-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rick o,5.0,Wonderful food with a delight to the eyes and ...
3,https://www.tripadvisor.com/Restaurant_Review-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rainb0wss,1.0,The staff who waited our table was friendly an...
4,https://www.tripadvisor.com/Restaurant_Review-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Thomas M,5.0,"Had eggs and a banana pancake soufflé,it was v..."
5,https://www.tripadvisor.com/Restaurant_Review-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,margie r,5.0,Step into this lovely garden that leads you in...
6,https://www.tripadvisor.com/Restaurant_Review-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Judy D,5.0,The Cream Pot now has floor to ceiling view wi...
7,https://www.tripadvisor.com/Restaurant_Review-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NashLuz,5.0,Had breakfast and tried soufflé pancakes . It ...
8,https://www.tripadvisor.com/Restaurant_Review-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tinar0s,5.0,This is a great place for breakfast and brunch...
9,https://www.tripadvisor.com/Restaurant_Review-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ashtahini,3.0,I came here with friends and we ordered a few ...


In [30]:
info_df_exploded = explode(info_df, lst_cols=['user_names', 'bubble_ratings', 'review_contents'])

ValueError: Length of values does not match length of index

In [ ]:
info_df_exploded.sample(5)

In [ ]:
info_df_exploded['location'][12]